In [31]:
import requests
import pandas as pd
import numpy as np
import json
from datetime import date

In [32]:
url = "https://covid-193.p.rapidapi.com/statistics"

In [33]:
headers = {
    'x-rapidapi-host': "covid-193.p.rapidapi.com",
    'x-rapidapi-key': "3ea1c14b00mshb6cedc9f4de354bp172633jsnc0ef76d6f7bd"
    }

In [34]:
result = requests.get(url, headers=headers)

In [35]:
pretty_json = json.loads(result.text)

In [36]:
all_df = pd.json_normalize(pretty_json['response'])

In [37]:
print(json.dumps(pretty_json, indent=2))

{
  "get": "statistics",
  "parameters": [],
  "errors": [],
  "results": 227,
  "response": [
    {
      "continent": "Asia",
      "country": "China",
      "population": 1439323776,
      "cases": {
        "new": "+12",
        "active": 184,
        "critical": 3,
        "recovered": 80566,
        "1M_pop": "59",
        "total": 85384
      },
      "deaths": {
        "new": null,
        "1M_pop": "3",
        "total": 4634
      },
      "tests": {
        "1M_pop": "111163",
        "total": 160000000
      },
      "day": "2020-09-29",
      "time": "2020-09-29T10:15:07+00:00"
    },
    {
      "continent": "Europe",
      "country": "Italy",
      "population": 60439986,
      "cases": {
        "new": "+1494",
        "active": 50323,
        "critical": 264,
        "recovered": 225190,
        "1M_pop": "5152",
        "total": 311364
      },
      "deaths": {
        "new": "+16",
        "1M_pop": "593",
        "total": 35851
      },
      "tests": {
        "1M

In [38]:
all_df.head()

,continent,country,population,day,time,cases.new,cases.active,cases.critical,cases.recovered,cases.1M_pop,cases.total,deaths.new,deaths.1M_pop,deaths.total,tests.1M_pop,tests.total
0,Asia,China,1.439324e+09,2020-09-29,2020-09-29T10:15:07+00:00,+12,184,3,80566,59,85384,None,3,4634,111163,160000000.0
1,Europe,Italy,6.043999e+07,2020-09-29,2020-09-29T10:15:07+00:00,+1494,50323,264,225190,5152,311364,+16,593,35851,184285,11138173.0
2,Europe,Spain,4.675921e+07,2020-09-29,2020-09-29T10:15:07+00:00,+2425,0,1529,0,16003,748266,+59,672,31411,272117,12723989.0
3,North-America,USA,3.314787e+08,2020-09-29,2020-09-29T10:15:07+00:00,+22,2542189,14065,4609636,22208,7361633,None,633,209808,317975,105401873.0
4,Europe,Germany,8.384995e+07,2020-09-29,2020-09-29T10:15:07+00:00,+127,26800,353,252400,3444,288745,None,114,9545,186555,15642654.0


In [39]:
all_df.shape

(227, 16)

In [40]:
cases_df = all_df.copy().drop(['tests.1M_pop', 
             'deaths.1M_pop', 
             'deaths.new', 
             'cases.recovered', 
             'tests.total', 
             'cases.critical', 
             'cases.active', 
             'cases.1M_pop',
             'cases.new',
             'time'                 
            ],
            axis = 1
           )

In [41]:
cases_df.dtypes

continent        object
country          object
population      float64
day              object
cases.total       int64
deaths.total      int64
dtype: object

In [42]:
cases_df.population = cases_df.population.fillna(0).astype(int)

In [43]:
cases_df

,continent,country,population,day,cases.total,deaths.total
0,Asia,China,1439323776,2020-09-29,85384,4634
1,Europe,Italy,60439986,2020-09-29,311364,35851
2,Europe,Spain,46759213,2020-09-29,748266,31411
3,North-America,USA,331478677,2020-09-29,7361633,209808
4,Europe,Germany,83849953,2020-09-29,288745,9545
...,...,...,...,...,...,...
222,None,Diamond-Princess,0,2020-09-29,712,13
223,Asia,Yemen,29980442,2020-09-29,2031,587
224,Asia,Tajikistan,9587978,2020-09-29,9685,75
225,Africa,Comoros,873862,2020-09-29,478,7


In [44]:
cases_europe_df = cases_df.loc[cases_df['continent'] == 'Europe'].reset_index().drop(['index', 'continent'], axis = 1)

In [45]:
cases_europe_df

,country,population,day,cases.total,deaths.total
0,Italy,60439986,2020-09-29,311364,35851
1,Spain,46759213,2020-09-29,748266,31411
2,Germany,83849953,2020-09-29,288745,9545
3,France,65308767,2020-09-29,542639,31808
4,Switzerland,8669932,2020-09-29,52646,2065
5,UK,67972591,2020-09-29,439013,42001
6,Netherlands,17144127,2020-09-29,114540,6380
7,Austria,9019005,2020-09-29,44041,796
8,Belgium,11602026,2020-09-29,115353,9987
9,Norway,5431601,2020-09-29,13788,274


In [46]:
if date.today().weekday() == 1:
        cases_YYYY_MM_DD = cases_europe_df.copy().drop(['deaths.total','day'], axis = 1)
        cases_YYYY_MM_DD = cases_YYYY_MM_DD.rename(columns={'cases.total' : f'{date.today()}'})
        